# Notificações
## Importando Bibliotecas

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML

In [6]:
results = pd.read_csv("compiled_result.csv")

In [7]:
# display(results)

In [8]:
results.drop(labels=['goal_type'], axis=1, inplace=True)

In [9]:
index_week = results.loc[results['goal'] == 19611, 'goal'].index.to_list()

## Separa os episódios em semanas

In [10]:
k = 0
week = []
for x in range(results.shape[0]):
    week.append(k)
    if x in index_week:
        k += 1
w = pd.Series(week, name= "week")
results = pd.concat([results, w], axis = 1)
results

,episode_id,team_id,player_id,indicator_id,result,score,goal,week
0,35,37,119,469,0.0,0.0,19611,0
1,35,37,119,470,0.0,NaN,0,1
2,35,37,119,471,0.0,NaN,0,1
3,35,37,119,472,0.0,NaN,0,1
4,35,37,119,473,0.0,NaN,0,1
...,...,...,...,...,...,...,...,...
995,22,24,110,263,21.0,NaN,0,100
996,22,24,110,264,203131.0,NaN,0,100
997,22,24,110,265,0.0,NaN,0,100
998,22,24,110,266,111.0,NaN,0,100


In [11]:
epi_unq = results['episode_id'].unique()

## Selecionando o episódio com ID 35, pois contém maior volume de dados

In [14]:
episode = results.loc[results['episode_id'] == epi_unq[0]]
episode

,episode_id,team_id,player_id,indicator_id,result,score,goal,week
0,35,37,119,469,0.0,0.0,19611,0
1,35,37,119,470,0.0,NaN,0,1
2,35,37,119,471,0.0,NaN,0,1
3,35,37,119,472,0.0,NaN,0,1
4,35,37,119,473,0.0,NaN,0,1
...,...,...,...,...,...,...,...,...
695,35,41,218,471,0.0,NaN,0,100
696,35,41,218,472,0.0,NaN,0,100
697,35,41,218,473,0.0,NaN,0,100
698,35,41,218,474,0.0,NaN,0,100


In [15]:
idc_unq = episode['indicator_id'].unique()
idc_unq

array([469, 470, 471, 472, 473, 474, 475])